In [ ]:
%%writefile simple_adder.sv

module simple_adder (
    input int a, b,
    output int c,

    input sub, clk, rst_n
);

always_ff @(posedge clk or negedge rst_n)
    if (!rst_n)   c <= 'd0;
    else if (sub) c <= a - b;
    else          c <= a + b;

endmodule


In [ ]:
%%writefile tb_simple_adder.cpp

#include <stdlib.h>
#include <iostream>
#include <cstdlib>
#include <verilated.h>
#include <verilated_vcd_c.h>
#include "Vsimple_adder.h"
#include "Vsimple_adder___024root.h"

#include <pybind11/pybind11.h>
namespace py = pybind11;

#define MAX_SIM_TIME 36
#define VERIF_START_TIME 7
vluint64_t sim_time = 0;

void do_subtraction (Vsimple_adder* dut, int a, int b) {
    dut->a = a;
    dut->b = b;
    dut->sub = 1;
    dut->eval();
    // Where should checking go??
    if (dut->c != a+b)
        printf("Error! Subtraction not correct, \
                a = %d, b = %d, c = %d, d = %d \n", \
                a, b, dut->c, a + b );
}

int main(int argc, char** argv, char** env) {
    srand (time(NULL));
    Verilated::commandArgs(argc, argv);
    Vsimple_adder *dut = new Vsimple_adder;

    Verilated::traceEverOn(true);
    VerilatedVcdC *m_trace = new VerilatedVcdC;
    dut->trace(m_trace, 5);
    m_trace->open("waveform.vcd");

    do_subtraction(dut, 7, 10);
 
    while (sim_time < MAX_SIM_TIME) {
        dut->clk ^= 1;
        dut->eval();
        printf("sim_time, rst_n is %ld, %d\n", sim_time, dut->rst_n);

        if (sim_time > 1 && sim_time < VERIF_START_TIME) {
            dut->rst_n = 0;
        }
        else {
            dut->rst_n = 1;
            if (dut->clk == 1)
                do_subtraction(dut, 7, 10);
        };

        m_trace->dump(sim_time);
        sim_time++;
    }
    
    m_trace->close();
    delete dut;
    exit(EXIT_SUCCESS);
}

In [ ]:
%%writefile simple_adder.cpp

#include <stdlib.h>
#include <iostream>
#include <cstdlib>
#include <verilated.h>
#include <verilated_vcd_c.h>
#include "Vsimple_adder.h"
#include "Vsimple_adder___024root.h"

#include <pybind11/pybind11.h>
namespace py = pybind11;

class SimpleAdder {
    public:
        SimpleAdder() {     // Constructor
            printf("Hello World!!\n");
            Verilated::traceEverOn(true);
            dut->trace(m_trace, 5);
            m_trace->open("waveform.vcd");
        }
        ~SimpleAdder() { 
            printf("Bye-Bye World!!\n");
            m_trace->close();
            delete dut;
            exit(EXIT_SUCCESS); 
        }   
        void initialize() {
            while (sim_time < 10) {
                dut->clk ^= 1;
                dut->eval();
                printf("sim_time, rst_n is %ld, %d\n", sim_time, dut->rst_n);

                if (sim_time > 1 && sim_time < 7) {
                    dut->rst_n = 0;
                }
                else {
                    dut->rst_n = 1;
                    if (dut->clk == 1)
                        printf("Will do a operation here...");
                };

                m_trace->dump(sim_time);
                sim_time++;
            }
        };
    private:
        vluint64_t sim_time = 0;
        VerilatedVcdC *m_trace = new VerilatedVcdC;
        Vsimple_adder* dut = new Vsimple_adder; 
};

PYBIND11_MODULE(simple_adder, m) {
    py::class_<SimpleAdder>(m, "SimpleAdder")
        .def(py::init<>())
        .def("initialize", &SimpleAdder::initialize);
}

In [ ]:
import os, sys

CONDA_PREFIX = os.environ.get('CONDA_PREFIX', None)
HOME = os.environ.get('HOME', None)
PATH = os.environ.get('PATH', None)

!make clean verilate
## YES THIS COMMAND SUCK -- CMAKE so it compiles to different objects
## Why C++ and not gcc/g++??
!c++ -O3 -Wall -shared -std=c++17 -fPIC `python3 -m pybind11 --includes` simple_adder.cpp obj_dir/*cpp \
$CONDA_PREFIX/share/verilator/include/verilated.cpp $CONDA_PREFIX/share/verilator/include/verilated_vcd_c.cpp \
-o simple_adder`python3-config --extension-suffix ` -I$CONDA_PREFIX/share/verilator/include -I$CONDA_PREFIX/share/verilator/include/vltstd -I./obj_dir -L./obj_dir/

In [ ]:
%%writefile simple_adder.py

import simple_adder

adder = simple_adder.SimpleAdder();
adder.initialize();


In [ ]:
!python simple_adder.py

In [ ]:
!make clean